# Eksplorujemy Bazę Danych: QDrant

* `QDrant` to baza danych, która pozwala na przechowywanie i przeszukiwanie wielowymiarowych wektorów. 
* Pozwalan na tzw. semantyczne wyszukiwanie czyli na podstawie znaczenia słów i fraz a nie tylko na podstawie ich wystąpień.
* `QDrant` jest dostępny jako usługa w chmurze, ale można go też zainstalować na własnym serwerze.
* `QDrant` posiada darmowy plan w chmurze!

## Jak zainstalować client do komunikacji z `QDrant`?

In [ ]:
# za pomocą conda (https://anaconda.org/conda-forge/qdrant-client)
!conda install -y qdrant-client

In [ ]:
# za pomoca pip
!pip install --quiet qdrant-client

In [ ]:
# do pracy z sekretami i kluczami
from dotenv import dotenv_values
# do pracy z qdrantem
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from qdrant_client.models import VectorParams, Distance
# do pracy z openai
from openai import OpenAI

In [ ]:
env = dotenv_values(".env")

In [ ]:
EMBEDDING_DIM = 1536

EMBEDDING_MODEL = "text-embedding-3-small"

def get_openai_client():
    return OpenAI(api_key=env["OPENAI_API_KEY"])

def get_embedding(text):
    openai_client = get_openai_client()
    result = openai_client.embeddings.create(
        input=[text],
        model=EMBEDDING_MODEL,
        dimensions=EMBEDDING_DIM,
    )

    return result.data[0].embedding

## Jak połączyć się z Qdrantem?

In [ ]:
# Tu jest zmiana względem wcześniejszej wersji
# Tworzymy klienta który komunikuje się z instancją bazy danych która znajduje się w chmurze
qdrant_client = QdrantClient(
    url=env["QDRANT_URL"],
    api_key=env["QDRANT_API_KEY"]
)

## Jak dodać dane do Qdranta?

In [ ]:
sci_fi_books = [
    {
        "name": "Wehikuł czasu",
        "description": "Człowiek podróżuje w czasie i jest świadkiem ewolucji ludzkości.",
        "author": "H.G. Wells",
        "year": 1895,
    },
    {
        "name": "Gra Endera",
        "description": "Młody chłopiec jest szkolony na dowódcę wojskowego w wojnie przeciwko obcej rasie.",
        "author": "Orson Scott Card",
        "year": 1985,
    },
    {
        "name": "Nowy wspaniały świat",
        "description": "Dystopijne społeczeństwo, w którym ludzie są genetycznie modyfikowani i warunkowani do przestrzegania ścisłej hierarchii społecznej.",
        "author": "Aldous Huxley",
        "year": 1932,
    },
    {
        "name": "Autostopem przez Galaktykę",
        "description": "Komediowa seria science fiction śledząca nieudane przygody nieświadomego człowieka i jego kosmicznego przyjaciela.",
        "author": "Douglas Adams",
        "year": 1979,
    },
    {
        "name": "Diuna",
        "description": "Planeta pustynna jest miejscem intryg politycznych i walk o władzę.",
        "author": "Frank Herbert",
        "year": 1965,
    },
    {
        "name": "Fundacja",
        "description": "Matematyk opracowuje naukę przewidywania przyszłości ludzkości i stara się uratować cywilizację przed upadkiem.",
        "author": "Isaac Asimov",
        "year": 1951,
    },
    {
        "name": "Zamieć",
        "description": "Futurystyczny świat, w którym internet przekształcił się w wirtualną rzeczywistość.",
        "author": "Neal Stephenson",
        "year": 1992,
    },
    {
        "name": "Neuromancer",
        "description": "Haker zostaje zatrudniony do wykonania niemal niemożliwego włamania i wplątuje się w sieć intryg.",
        "author": "William Gibson",
        "year": 1984,
    },
    {
        "name": "Wojna światów",
        "description": "Inwazja Marsjan na Ziemię rzuca ludzkość w chaos.",
        "author": "H.G. Wells",
        "year": 1898,
    },
    {
        "name": "Igrzyska śmierci",
        "description": "Dystopijne społeczeństwo, w którym nastolatkowie są zmuszani do walki na śmierć i życie w telewizyjnym spektaklu.",
        "author": "Suzanne Collins",
        "year": 2008,
    },
    {
        "name": "Szczep Andromedy",
        "description": "Śmiertelny wirus z kosmosu zagraża ludzkości.",
        "author": "Michael Crichton",
        "year": 1969,
    },
    {
        "name": "Lewa ręka ciemności",
        "description": "Ludzki ambasador zostaje wysłany na planetę, gdzie mieszkańcy są bezpłciowi i mogą zmieniać płeć według woli.",
        "author": "Ursula K. Le Guin",
        "year": 1969,
    },
    {
        "name": "Problem trzech ciał",
        "description": "Ludzie napotykają obcą cywilizację, która żyje w umierającym systemie.",
        "author": "Liu Cixin",
        "year": 2008,
    },
]

najpierw musimy stworzyć kolekcję. Kolekcja to zbiór tzw. punktów, które są wektorami w przestrzeni wielowymiarowej. Każdy punkt ma swój identyfikator, który jest unikalny w ramach kolekcji.

In [ ]:

QDRANT_COLLECTION_NAME = "sci_fi_books"

if not qdrant_client.collection_exists(collection_name=QDRANT_COLLECTION_NAME):
    print("Tworzę kolekcję")
    qdrant_client.create_collection(
        collection_name=QDRANT_COLLECTION_NAME,
        vectors_config=VectorParams(size=EMBEDDING_DIM, distance=Distance.COSINE),
    )

na koniec dodajemy punkty do kolekcji.

In [ ]:
for idx, book in enumerate(sci_fi_books)
    qdrant_client.upsert(
        collection_name=QDRANT_COLLECTION_NAME,
        points=[
            PointStruct(
                id=idx,
                vector=get_embedding(f'{book["name"]} {book["description"]} autorstwa: {book["author"]}'),
                payload=book
            )
        ]
    )

## Jak szukać punktów w Qdrancie?

In [ ]:
results = qdrant_client.search(
    collection_name=QDRANT_COLLECTION_NAME,
    query_vector=get_embedding("Inwazja kosmitów na Ziemię"),
    limit=3,
)
for result in results:
    print('TYTUŁ', result.payload["name"], 'OPIS', result.payload["description"], 'WYNIK', result.score)

## Budujemy funkcje: jak się upewnić, że kolekcja istnieje?

In [ ]:
def assure_db_collection_exists(
    qdrant_client,
    collection_name,
    embedding_dim,
):
    if not qdrant_client.collection_exists(collection_name):
        print(f"Tworzę kolekcję '{collection_name}'")
        qdrant_client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(
                size=embedding_dim,
                distance=Distance.COSINE,
            ),
        )
    else:
        print(f"Kolekcja '{collection_name}' już istnieje")

assure_db_collection_exists(qdrant_client, collection_name=QDRANT_COLLECTION_NAME, embedding_dim=EMBEDDING_DIM)